In [1]:
%load_ext autoreload

- This notebook is to explore what different AA models capture in texts when they are trained on the AA task (style?, topic?, discours?)
- We will use different AA models (StyleDistance, Wegmann, LUAR, etc.) and a sample from the HRS dataset.
- Method:
    - We cluster documents using the given AA model
    - Describe each cluster through a distribution over style, topic, dicourse, etc.
    - Quanitfy the alignment between these clusters and the corresponding distributions

- How to quantify the alignment?
    - Compute Avg similarity between distribution within the cluster vs. across the clusters
    - Compute correlation between cluster's distances and the corresponding representation 

In [2]:
import pandas as pd
import json
import sklearn
import glob
import pickle
import sys
import numpy as np
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from statsmodels.stats.dist_dependence_measures import distance_correlation
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [67]:
%autoreload
sys.path.append('../../style_generation_pipeline/')

import generate_explanations
from utils import *
from data import *

In [4]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_1/'

------------

In [191]:
def get_topic_representations(texts):
    topic_model = BERTopic(representation_model = KeyBERTInspired(), embedding_model="all-MiniLM-L6-v2")
    topic_model = topic_model.fit(texts)
    documents_props, _ = topic_model.approximate_distribution(texts)
    return documents_props

def get_cluster_topic_representations(cluster_df):
    document_texts = cluster_df.fullText.tolist()
    
    topic_model = BERTopic(representation_model = KeyBERTInspired(), embedding_model="all-MiniLM-L6-v2")
    topic_model = topic_model.fit(document_texts)
    
    cluster_to_documents_df = cluster_df.groupby('cluster_label').agg({'fullText': lambda x: list(x)}).reset_index()
    cluster_topic_reps = []
    for idx, row in cluster_to_documents_df.iterrows():
        documents_props, _ = topic_model.approximate_distribution(row['fullText'])
        #print(row['cluster_label'])
        #print(documents_props[0].shape)
        cluster_topic_reps.append(np.mean(documents_props, axis=0))

    return np.array(cluster_topic_reps)

def compute_style_overlap(interp_space, top_f=10):
    style_top_rep = [[y[0] for y in sorted(x[1][1].items(), key=lambda x:-x[1])[:top_f]] for x in interp_space.items()]
    perc_inters = 0
    total=0
    for i, rep1 in enumerate(style_top_rep):
        for j, rep2 in enumerate(style_top_rep):
            if i != j:
                reps_intersection = len(set(rep1).intersection(rep2))/(len(rep1) + len(rep2))
                #print(reps_intersection)
                #print(reps_intersection)
                perc_inters+= len(set(rep1).intersection(rep2))/(len(rep1) + len(rep2))
                total+=1

    return round(perc_inters/total, 3)

def analyze_correlation(model_interp_space, model_clustering_df, styles_df):
    # Compute latent, topic, and style representations
    sd_centroids  = np.array([val[0] for key, val in model_interp_space.items()])
    sd_style_dist = get_cluster_style_representations(styles_df, model_clustering_df)
    sd_topic_dist = get_cluster_topic_representations(model_clustering_df)

    # Compute pairwise distance according to each representation
    pairwise_style_dist  = cosine_distances(sd_style_dist)
    pairwise_interp_dist = cosine_distances(sd_centroids)
    pairwise_topic_dist  = cosine_distances(sd_topic_dist)

    style_corr= round(distance_correlation(pairwise_interp_dist, pairwise_style_dist), 2)
    topic_corr= round(distance_correlation(pairwise_interp_dist, pairwise_topic_dist), 2)
    # Compute the distance correlation
    print(style_corr, topic_corr)

    # Compute the style and topic representation for each document
    document_texts = model_clustering_df.fullText.tolist()
    document_ids   = model_clustering_df.documentID.tolist()
    doc_topic_reps = get_topic_representations(document_texts)
    doc_style_reps = get_document_style_representations(styles_df, document_ids)
    
    doc_topic_reps = {document_id: doc_topic_reps[i] for i, document_id in enumerate(document_ids)}
    doc_style_reps = {document_id: doc_style_reps[i] for i, document_id in enumerate(document_ids)}

    # Compute the average similarity between documents' representation in the same cluster
    avg_topic_sim = []
    avg_style_sim = []
    for cluster_id in model_clustering_df.cluster_label.unique():
        cluster_document_ids = model_clustering_df[model_clustering_df.cluster_label == cluster_id].documentID.tolist()
        other_document_ids   = model_clustering_df[model_clustering_df.cluster_label != cluster_id].documentID.tolist()
        #print(cluster_document_ids, other_document_ids)
        sim_between_clusters = np.mean(cosine_similarity([doc_topic_reps[i] for i in cluster_document_ids], [doc_topic_reps[i] for i in other_document_ids]))
        sim_within_cluster   = np.mean(cosine_similarity([doc_topic_reps[i] for i in cluster_document_ids]))
        avg_topic_sim.append(sim_within_cluster/sim_between_clusters)

        sim_between_clusters = np.mean(cosine_similarity([doc_style_reps[i] for i in cluster_document_ids], [doc_style_reps[i] for i in other_document_ids]))
        sim_within_cluster   = np.mean(cosine_similarity([doc_style_reps[i] for i in cluster_document_ids]))       
        avg_style_sim.append(sim_within_cluster/sim_between_clusters)

    avg_topic_sim = round(np.mean(avg_topic_sim), 2)
    avg_style_sim = round(np.mean(avg_style_sim), 2)
    
    print(avg_style_sim, avg_topic_sim)

    #compute top-style feats representation overlap between the clusters
    top_style_feats_overlap = compute_style_overlap(model_interp_space, top_f=10)
    
    return avg_topic_sim, topic_corr, avg_style_sim, style_corr, top_style_feats_overlap

### Generating clusters using Milad-TA2: 

In [137]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/training_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/valid_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/" \
--model aa_model-luar \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv" --top_k_feats 10 --eps 0.16

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Generating clusters using StyleDistance: 

In [19]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/training_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/valid_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/styledistance_explanations/" \
--model styledistance \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You try to use a model that was created with version 2.7.0, however, your version is 2.5.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



You try to use a model that was created with version 2.7.0, however, your version is 2.5.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



Testing Different Epsilon Values:   5%|5         | 5/99 [02:35<48:37, 31.04s/it]^C
Traceback (most recent call last):                                              
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 249, in <module>
    main(args)
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 103, in main
    best_eps, train_df = find_best_eps()
                         ^^^^^^^^^^^^^^^
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 90, in find_best_eps
    eps_to_performance[eps] = comput

In [134]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/training_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/valid_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/styledistance_explanations/" \
--model styledistance \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv" --top_k_feats 10 --eps 0.01

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You try to use a model that was created with version 2.7.0, however, your version is 2.5.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



You try to use a model that was created with version 2.7.0, however, your version is 2.5.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





### Generating clusters using Semantic Models (all-MiniLM-L6-v2): 

This is just to have a baseline for comparing. How much of the topic vs style correlation is there

In [25]:
# We can't find best clustering here, since we are comparing to ground-truth labels that reflect authorship attribution not topic. So let us just take a default value of 0.5
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/training_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/valid_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/semantic_explanations/" \
--model semantic_model \
--model_path "all-MiniLM-L6-v2" \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Testing Different Epsilon Values:   9%|9         | 9/99 [01:56<19:19, 12.89s/it]^C
Traceback (most recent call last):                                              
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 250, in <module>
    main(args)
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 103, in main
    best_eps, train_df = find_best_eps()
                         ^^^^^^^^^^^^^^^
  File "/home/ma4608/style_generation_pipeline/src-ipynb/../cluster_documents.py", line 90, in find_best_eps
    eps_to_performance[eps] = compute_model_performance(model, test_df, new_bases)
                              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ma4608/style_generation_pipeline/metrics/__init__.py", line 64, in compute_model_performance
    eer_interp.append(compute_eer(y_true, y_interp_score))
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ma4608/style_generat

In [138]:
! CUDA_VISIBLE_DEVICES=6 python ../cluster_documents.py --train-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/training_authors.json" \
--test-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/valid_authors.json" \
--save-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/semantic_explanations/" \
--model semantic_model \
--model_path "all-MiniLM-L6-v2" \
--style-dir "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv" --top_k_feats 10 --eps 0.5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### What do models capture?

In [139]:
styles_path = "/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/refined_and_aggregated_features_final.csv"
styles_df   = pd.read_csv(styles_path)
training_df = pd.read_json('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/training_candidates_and_queries.jsonl', lines=True)

model_clusterings = {
    'styledistance': ['/mnt/swordfish-pool2/milad/hiatus-data/phase_1/styledistance_explanations/'],
    'ta2_hrs': ['/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/'],
    'semantic': ['/mnt/swordfish-pool2/milad/hiatus-data/phase_1/semantic_explanations/'],
}

for key, val in model_clusterings.items():
    # style-distance clustering
    interp_space = pd.read_pickle(val[0] + "/interpretable_space.pkl")
    clustering_df  = pd.read_pickle(val[0] + "/training_authors_documents.pkl")
    # DBSCAN generate a cluster -1 of all outliers. We don't want this cluster
    del interp_space[-1]
    clustering_df = clustering_df[clustering_df.cluster_label != -1]
    
    model_clusterings[key].append(interp_space)
    model_clusterings[key].append(clustering_df)

In [189]:
model = 'ta2_hrs'
cluster_documents_df = model_clusterings[model][2].groupby('cluster_label').agg({'documentID': lambda x: len(list(x))}).reset_index()
interp_space_df = pd.DataFrame([{'cluster_label': x[0], 'style_feats': sorted(x[1][1].items(), key=lambda x:-x[1])} for x in model_clusterings[model][1].items()])
interp_space_df['num_feats'] = interp_space_df.style_feats.apply(lambda x: len(x))
interp_space_df['num_feats'] = interp_space_df.cluster_label.apply(lambda x: cluster_documents_df[cluster_documents_df.cluster_label==x]['documentID'].tolist()[0])

In [190]:
interp_space_df.head(n=10)

cluster_label  \
0              0   
1              1   
2              2   
3              3   
4              4   
5              5   
6              6   
7              7   
8              8   
9              9   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [140]:
#{x[0]: sorted(x[1][1].items(), key=lambda x:-x[1])[:10] for x in sd_interp_space.items()}
#sd_clustering.documentID.nunique()

In [195]:
# interp_space = model_clusterings['styledistance'][1]
# {x[0]: sorted(x[1][1].items(), key=lambda x:-x[1])[:10] for x in interp_space.items()}

In [196]:
models_analysis = {}

In [197]:
for model_name, model_interpretablity in model_clusterings.items():
    print(model_name, model_interpretablity[0])
    avg_topic_sim, topi_corr, avg_style_sim, style_corr = analyze_correlation(model_interpretablity[1], model_interpretablity[2], styles_df)
    models_analysis[model_name] = [avg_topic_sim, topi_corr, avg_style_sim, style_corr]

styledistance /mnt/swordfish-pool2/milad/hiatus-data/phase_1/styledistance_explanations/
0.43 0.69
Document 43d2e249-e16e-59e8-b271-4065734f42b0 doesn't exist!
Document f140817d-d77f-57cf-af98-b9e3bdefd608 doesn't exist!
2.05 1.76


ValueError: too many values to unpack (expected 4)

In [ ]:
print(tabulate(
        [[name] + scores for name, scores in models_analysis.items()],
        headers=['Embed Model', 'Avg. topic sim', 'Topic corr', 'Avg. style sim', 'Style corr']
    ))

In [13]:
print(tabulate(
        [[name] + scores for name, scores in models_analysis.items()],
        headers=['Embed Model', 'Avg. topic sim', 'Topic corr', 'Avg. style sim', 'Style corr']
    ))

Embed Model      Avg. topic sim    Topic corr    Avg. style sim    Style corr
-------------  ----------------  ------------  ----------------  ------------
styledistance              4.29          0.75              2.05          0.7
ta2_hrs                    5.93          0.81              2.38          0.77
semantic                   9.93          0.93              2.47          0.93


In [113]:
#docs_topic_reps = get_topic_representations(training_df.fullText.tolist())

In [148]:
#docs_style_reps = get_style_representation(styles_path, training_df.documentID.tolist())

In [115]:
#docs_latents, docs_interps = generate_explanations.get_documents_rep_vectors(training_df.fullText.tolist(), 'styledistance', 
#                                                               '/mnt/swordfish-pool2/milad/hiatus-data/phase_1/styledistance_explanations/interpretable_space.pkl')

In [21]:
# interp_space_path = '/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/interpretable_space.pkl'
# model_path = 'aa_model-luar'
# explanation_interf = explanation_interfaces.exp_interface_1
# input_path = '/mnt/swordfish-pool2/milad/hiatus-data/explainability-pilot-samples/'
# top_c=5